In [3]:
from data_loading import load_tumors
import glob
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline

GT_IMAGES_PATH = '/mnt/naruto/Seminar-Med-Seg2017/BRATS2017/HGG/Png-HGG/'
# Get results from directory when we stop woring on ground truth and know how to assign those images to our data
GAN_OUTPUT = '/mnt/naruto/Seminar-Med-Seg2017/SegMed/ultra/SegMed/pix2pix-ultimate/test-x/'
IMAGE_HEIGHT, IMAGE_WIDTH = 240, 240
TUMOR_IMAGES = 155

survivals = pd.read_csv("data/survival_data.csv")
survivals.head()

,Brats17ID,Age,Survival
0,Brats17_TCIA_167_1,74.907,153
1,Brats17_TCIA_242_1,66.479,147
2,Brats17_TCIA_319_1,64.860,254
3,Brats17_TCIA_469_1,63.899,519
4,Brats17_TCIA_218_1,57.345,346


In [4]:
def get_patient_images(paths, patient):
    # patient = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, TUMOR_IMAGES, 3))
    region_values = [211, 105, 240]
    for i in range(len(paths)):
        img = Image.open(paths[i]).convert('LA')
        img = np.asarray(img, dtype=np.uint8)[:, :, 0]
        for j in range(3):
            # The bottom layer contains the others and so on
            for layer in region_values[j:]:
                patient[i, j, :, :][img == layer] = 1
    return patient

def generate_testing_data(axis='z'):
    # x has shape (155, 240) -> use z for testing
    X = np.zeros((len(survivals), TUMOR_IMAGES, 3, IMAGE_HEIGHT, IMAGE_WIDTH), dtype=np.uint8)
    for index, row in tqdm(survivals.iterrows(), total=len(survivals)):
        # For every patient, fetch his tumor pictures and sort them properly
        glob_path = "{}{}_{}*n{}*.png".format(GT_IMAGES_PATH, row['Brats17ID'], 'seg', axis)
        tumor_images = glob.glob(glob_path)
        tumor_images.sort(key=lambda x : int(x.split('.')[-2]))
        # Convert every tumor image to greyscale and add it to X
        X[index, :, :, :, :] = get_patient_images(tumor_images, X[index, :, :, :, :])
    print('Resulting dataset has the shape: {}'.format(X.shape))
    np.save('data/tumors_n{}'.format(axis), X)
    return X

In [5]:
X = generate_testing_data()

100%|██████████| 163/163 [03:54<00:00,  1.43s/it]


Resulting dataset has the shape: (163, 155, 3, 240, 240)
